# Automated Email Marketing Part #3: Getting LLMs to Sell



In [ ]:
- Incorporate 
- Try different models
- Send off emails on its own

### What we did last time: Integrating Klaviyo with LangChain

In [6]:
import os
from dotenv import load_dotenv
from klaviyo_api import KlaviyoAPI

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [2]:
_ = load_dotenv()

In [3]:
# Extract Klaviyo (private) API key from .env file
klaviyo_key=os.getenv('KLAVIYO_API_KEY')

In [4]:
# Instantiate the Klaviyo Python API with the API key

klaviyo = KlaviyoAPI(klaviyo_key, max_delay=60, max_retries=3, test_host=None)

AttributeError: 'KlaviyoAPI' object has no attribute 'configuration'

In [ ]:
# Extract a sample of Metrics from the Klaviyo API

klaviyo.Metrics.get_metrics()

In [ ]:
# Store a sample customer ID from Klaviyo in an object

john_doe="01HVFKKSPHKNC5PV0VNN14TSJ4"

In [ ]:
# From here we can update a profile with custom properties

payload={ "data": {
        "type": "profile",
        "id": john_doe,
        "attributes": {
            'properties':{
                'clv':500,
                'churn_risk':'high',
                'upsales_propensity': '80%',
                'recommended_incentive': 'gift'
            }
                    }
            }
     }

klaviyo.Profiles.update_profile(john_doe, payload)

In [ ]:
# Extract events from Klaviyo for a given Klaviyo ID

events=klaviyo.Events.get_events(
    fields_event=['event_properties'],
    filter=f"equals(profile_id,'{john_doe}')",
    sort='-datetime'
    )

In [ ]:
# Extract profile data and events for a Klaviyo ID as a string we can feed to LangChain

def customer_data(custid):
    profile=klaviyo.Profiles.get_profile(custid)
    profile_string=str(profile['data']['attributes'])
    events=klaviyo.Events.get_events(fields_event=['event_properties'],
        filter=f"and(equals(profile_id,'{custid}'),equals(metric_id,'YtR57h'))",
        sort='-datetime')
    purchases='Purchases: '
    for e in events['data']:
        purchases=purchases+e['attributes']['event_properties']['Name']+','
    return profile_string+', '+purchases

In [ ]:
data=customer_data(john_doe)
data

In [ ]:
# Simple LLM chain that allows us to produce personalized text based on data from Klaviyo

prompt_template = """
Write a short 2 line introduction to an email with a hook from Max's Store to a customer with these {properties}
Don't mention the properties or the property values but try
to use the properties "upsales_propensity" and "recommended_incentive" in the formulation
"""

llm = ChatOpenAI(model_name="gpt-4", temperature=0.2)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
intro=llm_chain(data)

print(intro['text'])

### This time: Prompt Engineering in LangChain